In [1]:
import os
import numpy as np
import pandas as pd
from astropy.time import Time
from astropy import units as u
from validate_findorb.test_findorb import loadOrb,testFO,getOrbHorizons

# `validate_findorb` Demo
Here we are going to demo the basic functions of `validate_findorb`. We are going to show a general test case for validating the accuracy of find_orb with the main belt asteroid *202930 Ivezic (1998 SG172)*. `validate_findorb` and its testing function `testFO()` run on a simple testing structure.

Here are the general steps,

1. Take in initial orbit at a specified t0 time.
2. Generate Ephemeris observations using find_orb from the initial orbit over the time scale term supplied.
3. Determine the orbit from the Ephemeris observations with find_orb to get the orbital state vectors at the final time of the time scale.
4. Propagate the initial orbit using find_orb to the final time.
5. Compare the orbit's state vectors from step 3 to the orbit vectors from step 4.
---
## Importing Orbit Data
First we need to import the initial orbit data (state vector and time). We can use either `loadOrb()` to load from a local csv or pandas DataFrame or `getOrbHorizons()` to query JPL Horizons for the data. Both functions give the same orbit object.



In [3]:
#Importing Locally
orbits = loadOrb('orbits1.csv')
# Ivezic is the first object in the table
ivezic = orbits[0]
# We can show it as a DataFrame with .df attribute
ivezic.df

,orbit_id,mjd_tdb,x,y,z,vx,vy,vz,H,G
0,202930 Ivezic (1998 SG172),59000.000801,0.568687,-2.49376,-0.182298,0.010407,0.003676,0.000483,16.65,0.15


In [5]:
# Querying with JPL
# You have to specify a time for the position of the asteroid
# Using astropy.Time you can easily generate a time
t0 = Time([59000], scale="utc", format="mjd")
getOrbHorizons('202930',t0).df

,orbit_id,mjd_tdb,x,y,z,vx,vy,vz,H,G
0,202930 Ivezic (1998 SG172),59000.0,0.568687,-2.49376,-0.182298,0.010407,0.003676,0.000483,16.66,0.15


Now we have our orbit object for *202930 Ivezic (1998 SG172)* named `ivezic`. Now we can start running tests on this object.

---
## Generating Testing Parameters
Here we can generate a sample of parameters to test over. `validate_findorb` can only test over different observation time terms and astrometric errors. We can define the time frames using `np.arange()` which is in terms of days and the errors in terms of milliarcseconds.

In [12]:
dts = [
# Short-term, 0-30 days with an observation every 1 day
    np.arange(0, 30, 1),
    
# Weekly, 77 days with an observation every 7 days
    np.arange(0, 77, 7),
    
# Medium-term, 365 days with an observation every 10 days
    np.arange(0, 365 + 10, 10)
    
]

# 0 mas observation error, 10 mas error, and 100 mas error
errors = [0.,10.,100.]

## Using the Tester
Now that we have our parameters we can begin using the function `testFO()`. First we can define the initial time `t0` as the time when our initial orbits are located, thus we can use `ivezic.epochs`. Then we can define the `observatory_code` as "500" which is the Geocenter. In the future different MPC observatory code can be used for this parameter.

Now we will use the first time scale and first astrometric error we defined above.

In [9]:
t0 = ivezic.epochs
testFO(ivezic,'500',t0,dts[0],errors[0])

,orbit_id,observatory_code,arc_length [days],num_obs,num_obs_fit,epoch [mjd],astrometric_error [mas],delta epoch [mjd],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s],rms delta ra [arcsec],rms delta dec [arcsec],rms delta time [seconds],covariance
0,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,0.0,9.997748e-07,0.000046,1.589317e-08,0.000021,-0.00004,-0.000005,-8.887649e-09,1.307467e-08,1.629640e-09,5.558549e-10,1.716718e-10,0.0,"[[1.72269778462e-08, -3.19807119244e-08, -3.99..."


Thus `testFO()` returns the difference DataFrame which can be saved and analyzed over. See the documentation for a description of each column.

---

## Varying Time Terms
Here we can vary the time scales for each by giving `testFO()` the list of time scales `dts` we defined above.

In [10]:
testFO(ivezic,'500',t0,dts,errors[0])

,orbit_id,observatory_code,arc_length [days],num_obs,num_obs_fit,epoch [mjd],astrometric_error [mas],delta epoch [mjd],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s],rms delta ra [arcsec],rms delta dec [arcsec],rms delta time [seconds],covariance
0,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,0.0,9.997748e-07,0.000046,1.589317e-08,0.000021,-0.000040,-5.057349e-06,-8.887649e-09,1.307467e-08,1.629640e-09,5.558549e-10,1.716718e-10,0.0,"[[1.72269778462e-08, -3.19807119244e-08, -3.99..."
1,202930 Ivezic (1998 SG172),500,70.0,11,11,59070.0008,0.0,9.997748e-07,0.000009,1.037233e-09,-0.000003,0.000008,7.432393e-07,8.109717e-10,-6.457737e-10,-3.397821e-11,1.905235e-09,6.632126e-10,0.0,"[[1.70555156196e-09, -4.71749358485e-09, -4.89..."
2,202930 Ivezic (1998 SG172),500,370.0,38,38,59370.0008,0.0,9.997748e-07,0.000280,1.257969e-08,0.000189,-0.000205,-1.919551e-05,1.233878e-08,-2.435919e-09,2.631903e-10,4.881165e-09,1.556552e-09,0.0,"[[2.55722264131e-11, 8.61391086906e-12, 1.9792..."


## Varying Time and Error Terms
We can vary the time scales and error terms by passing both lists (`dts` and `errors`) to `testFO()`,

In [11]:
testFO(ivezic,'500',t0,dts,errors)

,orbit_id,observatory_code,arc_length [days],num_obs,num_obs_fit,epoch [mjd],astrometric_error [mas],delta epoch [mjd],delta r [km],delta v [m/s],delta x [km],delta y [km],delta z [km],delta vx [m/s],delta vy [m/s],delta vz [m/s],rms delta ra [arcsec],rms delta dec [arcsec],rms delta time [seconds],covariance
0,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,0.0,9.997748e-07,0.000046,1.589317e-08,0.000021,-0.000040,-5.057349e-06,-8.887649e-09,1.307467e-08,1.629640e-09,5.558549e-10,1.716718e-10,0.000000,"[[1.72269778462e-08, -3.19807119244e-08, -3.99..."
1,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,10.0,9.997748e-07,1118.260575,5.357818e-01,522.823189,-981.280892,-1.193752e+02,-2.972887e-01,4.427592e-01,5.143866e-02,1.145374e-02,7.393067e-03,3.530087,"[[1.96342482582e-11, -3.64497440518e-11, -4.55..."
2,202930 Ivezic (1998 SG172),500,29.0,30,30,59029.0008,100.0,9.997748e-07,9809.779807,8.900815e+00,4537.268933,-8636.756271,-1.025384e+03,-4.743230e+00,7.474880e+00,9.232779e-01,9.989134e-02,7.344748e-02,37.182203,"[[1.74900190544e-09, -3.24691966198e-09, -4.05..."
3,202930 Ivezic (1998 SG172),500,70.0,11,11,59070.0008,0.0,9.997748e-07,0.000009,1.037233e-09,-0.000003,0.000008,7.432393e-07,8.109717e-10,-6.457737e-10,-3.397821e-11,1.905235e-09,6.632126e-10,0.000000,"[[1.70555156196e-09, -4.71749358485e-09, -4.89..."
4,202930 Ivezic (1998 SG172),500,70.0,11,11,59070.0008,10.0,9.997748e-07,447.667586,3.332501e-02,149.588168,-419.487892,-4.538233e+01,-2.988901e-02,1.411840e-02,4.227730e-03,9.478347e-03,9.023498e-03,2.386629,"[[1.85132217294e-12, -5.12066313595e-12, -5.31..."
5,202930 Ivezic (1998 SG172),500,70.0,11,11,59070.0008,100.0,9.997748e-07,2294.842870,1.221181e-01,-668.973900,2186.435492,1.956465e+02,1.215066e-01,9.495207e-03,-7.669131e-03,7.665615e-02,1.014188e-01,20.712952,"[[1.78384981137e-10, -4.93403752848e-10, -5.11..."
6,202930 Ivezic (1998 SG172),500,370.0,38,38,59370.0008,0.0,9.997748e-07,0.000280,1.257969e-08,0.000189,-0.000205,-1.919551e-05,1.233878e-08,-2.435919e-09,2.631903e-10,4.881165e-09,1.556552e-09,0.000000,"[[2.55722264131e-11, 8.61391086906e-12, 1.9792..."
7,202930 Ivezic (1998 SG172),500,370.0,38,38,59370.0008,10.0,9.997748e-07,42.505178,3.041335e-03,42.101483,5.549378,-1.832946e+00,2.975050e-03,6.312595e-04,1.756852e-05,1.119984e-02,1.131512e-02,2.669847,"[[3.61433007533e-14, 1.21722643358e-14, 2.7961..."
8,202930 Ivezic (1998 SG172),500,370.0,38,38,59370.0008,100.0,9.997748e-07,110.221830,1.243743e-02,57.155091,93.270656,-1.351785e+01,3.636635e-03,1.183448e-02,1.187200e-03,8.222673e-02,9.394407e-02,15.377388,"[[2.61879751063e-12, 8.82133254914e-13, 2.0263..."


Wonderful! You have successfully used the tester. 

This tester can also be used for multiple asteroid orbits and the outputs will scale accordingly. For the sake of not knowing how much computing power and time the user has, I will not be demonstrating this feature here. Try running the `orbits` object above in `testFO()` for a demo.

---
## Command and Bash Script Output
You can use the `out` parameter to save all the files generated by the tester and the relevant command line calls to a local directory so the calls to find_orb can be run independently from the tester. This includes generated Bash scripts for ease of use.

---
If you have any more questions, email me aidan@b612foundation.org